# Import, and analyze audio files

In [1]:
import os
import statistics
import numpy as np
from matplotlib import pyplot as plt
import IPython.display as ipd
import librosa, librosa.display
import pandas as pd
import math
import ffmpeg
import pydub
import csv
import sklearn
import sounddevice as sd#to play back sounds
from pydub import AudioSegment
from scipy import signal#for fast correlation (using fast convolution)
from scipy.signal import butter, lfilter, freqz, sosfreqz, sosfilt
from scipy.signal import find_peaks
import scipy

In [2]:
# This is a band pass filter:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    sos = butter(order, [low, high], btype='band',output='sos')
    return sos

def butter_bandpass_filter(data, lowcut, highcut, sr, order=5):
    sos = butter_bandpass(lowcut, highcut, sr, order=order)
    y = sosfilt(sos, data)
    return y

In [3]:
# Extraction  of all recording studio features per frame
def recordingstudio(cut, fs):
    bc = []
    peaks = []
    rmss = []
    crests = []
    corr=[]
    line=[]
    #klasse=[]
    for frame in range(int(len(cut[0])/fs)):
        box = [[0,0] for i in range(fs)]
        left = [0 for i in range(fs)]
        right = left
        for i in range(fs):
            left[i]=cut[0][frame*fs+i]
            right[i]=cut[1][frame*fs+i]
            box[i]=[int(math.floor(left[i]*20)),int(math.floor(right[i]*20))]
            sq = left[i]**2
        peak=np.max(np.abs(left))
        if peak < 0.000001:
            peak = 0.000001
        #print(peak)
        corr.append(np.min(np.corrcoef(cut[0][frame*fs:frame*fs+fs],cut[1][frame*fs:frame*fs+fs])))
        rms=np.sqrt(np.mean(sq))#0 to 1
        if rms < 0.0000002:
            rms = 0.0000002
        peakdB=20*math.log(peak,10)#-infty to 0
        rmsdB=20*math.log(rms,10)#0 to -infty
        crest=peakdB-rmsdB
        bc.append(len(np.unique(box))/400)#0.0025 to 1
        peaks.append(peakdB)
        rmss.append(rmsdB)
        crests.append(crest)#0 to 80.9
        line.append(frame)
        #klasse.append(1)
    #print('all frames done')
    return line, bc, corr, peaks, rmss, crests#,class 
#print(type(line))

In [4]:
# define source folder:
mp3list = []
path = r'C:\Users\t_ziemer\Desktop\techno-94\usa2025'#file path that contains all mp3 files to be analyzed
for root, dirs, files in os.walk(path):#loop to list all file paths
    for file in files:
        mp3list.append(os.path.join(root,file))
fs = 2048#frame size about 50 ms
ss = fs#step size equals frame size (i.e., no overlap between frames)

In [5]:
# loop through, export files and skip corrupted files
import warnings
# Suppress DeprecationWarning
warnings.simplefilter("ignore", category=DeprecationWarning)
for n in range(len(mp3list)):#len(mp3list)
    while True:
        try:
            #dur = pydub.utils.mediainfo(mp3list[n])["duration"]
            array, sr=librosa.load(mp3list[n],sr=44100,mono=False)#,duration = math.floor(float(dur))
            print(mp3list[n])
            #normalization:
            if np.max(array)>np.min(array)*-1:
                array = array/np.max(array)
            else:
                array = array/np.min(array)*-1
            cut = [array[0][176400:len(array[0])-176400],array[1][176400:len(array[1])-176400]]# we crop the first and last 4 seconds
            line, bc, corr, peak, rms, crest = recordingstudio(cut, fs)
            cutlow = [butter_bandpass_filter(cut[0],20, 150 ,sr, order=5),butter_bandpass_filter(cut[1],20, 150 ,sr, order=5)]
            line, bclow, corrlow, peaklow, rmslow, crestlow = recordingstudio(cutlow, fs)
            cutmid = [butter_bandpass_filter(cut[0],150, 2000 ,sr, order=5),butter_bandpass_filter(cut[1],150, 2000 ,sr, order=5)]
            line, bcmid, corrmid, peakmid, rmsmid, crestmid = recordingstudio(cutmid, fs)
            cuthigh = [butter_bandpass_filter(cut[0],2000, 10000 ,sr, order=5),butter_bandpass_filter(cut[1],2000, 10000 ,sr, order=5)]
            line, bchigh, corrhigh, peakhigh, rmshigh, cresthigh = recordingstudio(cuthigh, fs)
            tempo, beat_frames = librosa.beat.beat_track(y=cut[0], sr=sr)
            beats=[]            
            if tempo < 100:
                tempo = tempo * 2
            if tempo > 220:
                tempo = tempo / 2
            for i in range(len(line)):
                beats.append(tempo)#bpm,median
            #cutleft= cut[0][0::32]
            #estimated_bpm = estimate_bpm(cutleft)
            #median=np.median(estimated_bpm)
            features=[np.transpose(line), np.transpose(bc), np.transpose(corr), np.transpose(peak), np.transpose(rms), np.transpose(crest), np.transpose(bclow), np.transpose(corrlow), np.transpose(peaklow), np.transpose(rmslow), np.transpose(crestlow), np.transpose(bcmid), np.transpose(corrmid), np.transpose(peakmid), np.transpose(rmsmid), np.transpose(crestmid), np.transpose(bchigh), np.transpose(corrhigh), np.transpose(peakhigh), np.transpose(rmshigh), np.transpose(cresthigh),np.transpose(beats)] 
            Details = ["", 'PhaseSpace', 'ChannelCorrelation', 'PeakMeter', 'RMS', 'CrestFactor','PhaseSpaceLow', 'ChannelCorrelationLow', 'PeakMeterLow','RMSLow','CrestFactorLow', 'PhaseSpaceMid', 'ChannelCorrelationMid', 'PeakMeterMid','RMSMid','CrestFactorMid', 'PhaseSpaceHigh', 'ChannelCorrelationHigh', 'PeakMeterHigh','RMSHigh','CrestFactorHigh','bpm']
            with open(mp3list[n]+'.csv', 'w', newline='') as f:
                write = csv.writer(f) 
                write.writerow(Details) 
                write.writerows(np.transpose(features)) 
            break      
        except:
            print('shit!')
            break

C:\Users\t_ziemer\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


C:\Users\t_ziemer\Desktop\techno-94\usa2025\A Nite Life Production Presents Smooth Stan Smith - On My Own (1994).m4a


C:\Users\t_ziemer\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:2853: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]


C:\Users\t_ziemer\Desktop\techno-94\usa2025\A Nite Life Production Presents Smooth Stan Smith - Swing Side (1994).opus
C:\Users\t_ziemer\Desktop\techno-94\usa2025\A Nite Life Production Presents Smooth Stan Smith - U Flat (1994).opus


C:\Users\t_ziemer\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


C:\Users\t_ziemer\Desktop\techno-94\usa2025\Alpha Team - Speed (Club Mix) (1992.m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Alpha Team - Speed (Hardcore) (1992).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Andre Wade & Company - Bend Over (PJ's Underground Mix) (1993).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Bobby D - Edit Crazy #0 (1988).m4a


C:\Users\t_ziemer\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:2853: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\t_ziemer\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


C:\Users\t_ziemer\Desktop\techno-94\usa2025\La Drid Austin - Don't Make Me Wait (1992).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\La Drid Austin - Flashback (1992).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\La Drid Austin - Outrage (1992).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\La Drid Austin - Strange Invader (1992).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\La Drid Austin - You Got To Move (1992).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Lydia Lee - I Do Love You (Club Mix) (1991).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\M.T.S. - Time To Overload (Rx 1990) (1990).opus
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Methodology - Rockin To The Bass (1992).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Methodology - Take A Ride (1992).m4a
shit!
shit!
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Paul Johnson - Drum Boogie (1992).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Paul Johnson - Fascination (1992).m4a


C:\Users\t_ziemer\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:2853: RuntimeWarning: divide by zero encountered in divide
  c /= stddev[:, None]
C:\Users\t_ziemer\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: divide by zero encountered in divide
  c /= stddev[None, :]


C:\Users\t_ziemer\Desktop\techno-94\usa2025\Paul Johnson - Kaotil Samples (1992).m4a
shit!
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Paul Johnson - Love Feeling (1992).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Paul Johnson - Men at Work (1992).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Paul Johnson - Over Load #12 (1992).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Paul Johnson - Plus Times Minus (1992).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Paul Johnson - S.O.S. (1992).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Paul Johnson - Time Warp (1993).m4a
shit!
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Ron Carroll - A New Day (P.J. Dub) (1993).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Smooth Stan Smith - 2 Smooth (1994).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Two City Tribe - Move Something (Radio Mix) (1992).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Unisyn - Take The Time (Get Downs Dope Mix) (1992).m4a
C:\Users\t_ziemer\Desktop\techno-94\us